In [1]:
import pandas as pd
import numpy as np
import json
import re

In [5]:


products_df = pd.read_csv('../data/amazon_meta.csv')
reviews_df = pd.read_csv('../data/amazon_reviews.csv')

# print(df.head())

# print(df['details'].iloc[0])

# print(df.columns)

In [6]:
print(products_df.columns)
print(products_df.head().to_markdown())

Index(['Unnamed: 0', 'main_category', 'title', 'average_rating',
       'rating_number', 'features', 'description', 'price', 'images', 'videos',
       'store', 'categories', 'details', 'parent_asin', 'bought_together',
       'subtitle', 'author'],
      dtype='object')


ImportError: Missing optional dependency 'tabulate'.  Use pip or conda to install tabulate.

In [ ]:
print(reviews_df.columns)
print(reviews_df.head().to_markdown())

In [ ]:
missing = reviews_df.isna().sum()
missing_percent = (missing / len(reviews_df)) * 100
missing_summary = pd.DataFrame({
    "missing_count": missing,
    "missing_%": missing_percent.round(2)
}).sort_values(by="missing_%", ascending=False)
print("\n Missing Values Summary:")

display(missing_summary)


 Missing Values Summary:


,missing_count,missing_%
title,804,0.02
text,1033,0.02
Unnamed: 0,0,0.00
rating,0,0.00
images,0,0.00
asin,0,0.00
parent_asin,0,0.00
user_id,0,0.00
timestamp,0,0.00
helpful_vote,0,0.00


In [ ]:
# 50% of products missing price
missing = products_df.isna().sum()
missing_percent = (missing / len(products_df)) * 100
missing_summary = pd.DataFrame({
    "missing_count": missing,
    "missing_%": missing_percent.round(2)
}).sort_values(by="missing_%", ascending=False)
print("\n Missing Values Summary:")
display(missing_summary)


 Missing Values Summary:


,missing_count,missing_%
bought_together,137269,100.00
author,137007,99.81
subtitle,136919,99.75
price,75261,54.83
main_category,11036,8.04
store,4375,3.19
title,9,0.01
Unnamed: 0,0,0.00
average_rating,0,0.00
images,0,0.00


In [ ]:
# TODO:
# Data Cleaning Steps:
# Change na values to empty strings in product titles in product_reviews
# Change na values to empty strings in store, titles, and main_categorey in products_df

# Impute median value for missing prices in products_df
# Get rid of bought_togher, author, subtitle in products_df

# Join on parent_asin 
# save this combined dataframe to csv

In [ ]:
# TODO:
# Recommender System Steps:
# Build item representation
# Follow steps similar to assign 3 notebook 4 
# EX: make all strings into a vector representation using TFIDF/BM25 
# EX: add SCALED numerical features like price, review ratings, etc.
# Compute item to item similarity matrix


In [ ]:
products_df.dtypes
products_df.head(1)

,Unnamed: 0,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author,merged_text
0,0,Video Games,Dash 8-300 Professional Add-On,5.0,1,"[""Features Dash 8-300 and 8-Q300 ('Q' rollout ...",['The Dash 8-300 Professional Add-On lets you ...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Aerosoft,"['Video Games', 'PC', 'Games']",{'Pricing': 'The strikethrough price is the Li...,B000FH0MHO,NaN,NaN,NaN,0 Video Games Dash 8-300 Professional Add-On 5...


Clean the dataframes

In [ ]:
def convert_price_to_int(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip().lower()
    if s in {'', 'nan', 'none', 'n/a', 'free'}:
        return np.nan
    # remove common currency symbols and thousands separators
    s = re.sub(r'[\$€,£¥]', '', s).replace(',', '')
    nums = re.findall(r'\d*\.\d+|\d+', s)
    if not nums:
        return np.nan
    nums = [float(n) for n in nums]
    return float(sum(nums) / len(nums))  # mean if multiple, single value otherwise


products_df['price'] = products_df['price'].apply(convert_price_to_int)

In [ ]:
reviews_df['title'] = reviews_df['title'].fillna('')
reviews_df['text'] = reviews_df['text'].fillna('')


products_df['store'] = products_df['store'].fillna('')
products_df['title'] = products_df['title'].fillna('')
products_df['main_category'] = products_df['main_category'].fillna('')

median_price = products_df['price'].median().astype(float)
products_df['price'] = products_df['price'].fillna(median_price)
cleaned_products = products_df.drop(columns=['bought_together', 'author', 'subtitle'])




In [ ]:

def row_to_text(row):

    parts = []
    for value in row:
        if pd.isna(value):
            continue
        if isinstance(value, (list, dict)):
            try:
                value = json.dumps(value)
            except:
                value = str(value)

        parts.append(str(value))

    return " ".join(parts)


cleaned_products["merged_text"] = cleaned_products.apply(row_to_text, axis=1)



In [ ]:
reviews_df.groupby('parent_asin').size().sort_values(ascending=False).head(10)


parent_asin
B01N3ASPNV    18105
B0BN942894    17310
B077GG9D5D    15594
B000N5Z2L4    13329
B0086VPUHI    12100
B004RMK5QG    10256
B07YBXFDYN     9430
B00BGA9WK2     8089
B07V8YSBFG     7844
B07CRX5X9T     7570
dtype: int64

: 

In [20]:
combined_df = pd.merge(reviews_df, cleaned_products, left_on='parent_asin', right_on='parent_asin', how='inner')



combined_df.to_csv('../data/combined_amazon_data.csv', index=False)


In [2]:
combined_df = pd.read_csv('../data/combined_amazon_data.csv')


KeyboardInterrupt: 

In [3]:
combined_df = pd.read_csv('../data/combined_amazon_data.csv', nrows=100)


In [4]:
combined_df.head(2)

,Unnamed: 0_x,rating,title_x,text,images_x,asin,parent_asin,user_id,timestamp,helpful_vote,...,rating_number,features,description,price,images_y,videos,store,categories,details,merged_text
0,0,4.0,It’s pretty sexual. Not my fav,I’m playing on ps5 and it’s interesting. It’s...,[],B07DJWBYKP,B07DK1H3H5,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,1608186804795,0,...,2015,['Become a cyberpunk an urban mercenary equipp...,"[""Cyberpunk 2077 is an open world, an action a...",24.95,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Cyberpunk 2077 gameplay', 'url': '...",WARNER BROS,"['Video Games', 'PC', 'Games']","{'Release date': 'December 10, 2020', 'Best Se...",98979 Video Games Cyberpunk 2077 - PC [Game Do...
1,1,5.0,Good. A bit slow,Nostalgic fun. A bit slow. I hope they don’t...,[],B00ZS80PC2,B07SRWRH5D,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,1587051114941,1,...,9097,['Explore a dark & eclectic world – Dive deep ...,['A spectacular reimagining of one of the most...,25.95,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Watch before you play this game!',...",Square Enix,"['Video Games', 'PlayStation 4', 'Games']","{'Release date': 'April 10, 2020', 'Best Selle...",44810 Video Games Final Fantasy VII: Remake - ...
